## test code part 1

In [1]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain_intro.chatbot import chat_model

In [2]:
messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about
        healthcare. Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is Medicaid managed care?"),
]

In [3]:
chat_model.invoke(messages)

AIMessage(content='Medicaid managed care is a system in which state Medicaid programs contract with private health insurance companies to provide healthcare services to Medicaid beneficiaries. In this model, the managed care organizations (MCOs) are responsible for coordinating and delivering a range of healthcare services, including preventive care, hospital services, and specialty care, to enrolled members.\n\nKey features of Medicaid managed care include:\n\n1. **Care Coordination**: MCOs help coordinate care among various providers to ensure that beneficiaries receive appropriate and timely services.\n\n2. **Cost Control**: By managing the care provided to beneficiaries, MCOs aim to control costs and improve efficiency in the delivery of healthcare services.\n\n3. **Access to Services**: Beneficiaries typically have a network of providers from which to choose, and MCOs may offer additional services beyond what traditional Medicaid covers.\n\n4. **Quality Improvement**: Many MCOs im

In [4]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDid anyone have a positive experience?\n"

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)
context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive experience?')]

## test code part 2

In [1]:
from langchain_intro.chatbot import review_chain

context = "I had a great stay!"
question = "Did anyone have a positive experience?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, one patient mentioned that they had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 83, 'total_tokens': 103}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None}, id='run-9e440f5f-9648-454b-b1c7-1f4e99719b65-0')

## test code 3

In [37]:
import pandas as pd
from uuid import uuid4
import dotenv
from langchain_openai import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_core.documents import Document



In [45]:
# Load environment variables
dotenv.load_dotenv()

# File paths and other configurations
REVIEWS_CSV_PATH = "data/reviews.csv"
ELASTICSEARCH_URL = "http://localhost:9200"
INDEX_NAME = "langchain_index"

# Load CSV data with pandas
df = pd.read_csv(REVIEWS_CSV_PATH, encoding='utf-8')

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Configure ElasticsearchStore
elastic_vector_search = ElasticsearchStore(
    es_url=ELASTICSEARCH_URL,
    index_name=INDEX_NAME,
    embedding=embeddings,
    es_user="elastic",
    es_password="changeme"
)


In [55]:
# Create Document objects
documents = []
uuids = []

for _, row in df.iterrows():
    doc_id = str(uuid4())
    documents.append(Document(
        page_content=row["review"].encode("utf-8"),  # Use the "review" column for content
        metadata={
            "review_id": row["review_id"],
            "visit_id": row["visit_id"],
            "physician_name": row["physician_name"].encode('utf-8'),
            "hospital_name": row["hospital_name"].encode('utf-8'),
            "patient_name": row["patient_name"].encode('utf-8')
        }
    ))
    uuids.append(doc_id)




In [63]:
try:
    elastic_vector_search.add_documents(docs)
    print("Documents have been indexed successfully.")
except AttributeError as e:
    print(f"AttributeError: {e}")
except UnicodeEncodeError as e:
    print(f"UnicodeEncodeError: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

UnicodeEncodeError: 'ascii' codec can't encode character '\u201c' in position 7: ordinal not in range(128)


In [41]:
documents[2]

Document(metadata={'review_id': 11, 'visit_id': 680, 'physician_name': 'Chase Mcpherson Jr.', 'hospital_name': 'Wallace-Hamilton', 'patient_name': 'Abigail Mitchell'}, page_content="The hospital's commitment to patient safety was evident throughout my stay. The stringent hygiene protocols and vigilant staff instilled confidence in the quality of care provided.")

In [57]:
from langchain.document_loaders import CSVLoader
loader = CSVLoader('data/reviews.csv', encoding='utf8')

In [62]:
docs = loader.load()